# Player Historical Performance Scrawler
- Detailed performance (goals, assistants, passes, etc.)
  - webpage to SQL (Selenium)

In [1]:
import numpy, time, random
from bs4 import BeautifulSoup
import pandas as pd
from sqlalchemy import create_engine
from selenium import webdriver as wd
from datetime import datetime


In [2]:
# load existing PlayerList db
disk_engine = create_engine('sqlite:///2018WC_ZL_test2.db')
player_name_df = pd.read_sql('PlayerList', disk_engine)
match_list_df = pd.read_sql('MatchList', disk_engine)
player_data_df = pd.read_sql('PlayerData', disk_engine)
player_data_goalkeeper_df = pd.read_sql('PlayerData_GoalKeeper', disk_engine)
# Start selenium
driver = wd.Firefox(executable_path='C:\Program Files\geckodriver\geckodriver')



In [6]:
# Loop through players, retrive performance data, and store in SQL db
number_of_nan_roto_ID = 0
for roto_ID in player_name_df['roto_Player_ID']:  # loop through all players
    roto_ID = int(roto_ID)
    if player_data_df['roto_Player_ID'].isin([roto_ID]).any() | player_data_goalkeeper_df['roto_Player_ID'].isin(
            [roto_ID]).any():
        # check if current player has already been processed -- if yes, skip
        print('roto player', player_name_df[player_name_df['roto_Player_ID'] == roto_ID]['roto_Name'].iloc[0],
              'already in player_data_df, continue')
        continue
    print('')
    print('working on roto player', player_name_df[player_name_df['roto_Player_ID'] == roto_ID]['roto_Name'].iloc[0])

    # Check roto_ID, there is one player in DB does not have rotopage, continue in that case
    if numpy.isnan(roto_ID):
        if number_of_nan_roto_ID > 1:  # more than one player not in the DB, raise error
            raise Exception('number of NaNs in db > 1 -- should only be Kenner Gutierrez from CRC')
        number_of_nan_roto_ID = number_of_nan_roto_ID + 1
        continue

    # use selenium to get page source -- regular page source from urllib filled up with extra HTTP call and
    # javascript code being executed on the browser side
    roto_link = 'https://www.rotowire.com/soccer/player.php?id=' + str(int(roto_ID))
    driver.get(roto_link)
    time.sleep(random.uniform(1.5, 2.5))  # wait a second until page load

    # Click on DraftKings in Scoring at the first page
    try:
        is_DraftKings_clicked # do nothing if already clicked
    except NameError:
        element = driver.find_element_by_xpath('//div[(@class="toggle-tab") and (@data-name = "DraftKings")]')
        element.click()
        is_DraftKings_clicked = True

    # Double Click League to make World Cup matches visible
    time.sleep(random.uniform(1.5, 2.5))  # wait a second until page load
    element = driver.find_element_by_xpath('//div[@role="columnheader" and @class="webix_hcell align-c"]')
    element.click()
    time.sleep(random.uniform(1,1.5))  # wait a second until page load
    element.click()

    # load page to beautiful soup
    player_page_source = driver.page_source
    player_soup = BeautifulSoup(player_page_source, 'html.parser')

    # find data headers
    player_data_headers = [x.contents[0] for x in player_soup.find_all('div', {'class': 'webix_ss_header'})[3] \
        .find_all('div', {'role': 'columnheader'})]
    if len(player_data_headers) == 29:
        is_goally = False
    elif len(player_data_headers) == 14:
        is_goally = True
    else:
        raise Exception('player_data_headers has wrong number of contents')

    # Process player data
    curr_player_data_list = []  # list to store match by match player data, before storing in dataframe
    player_data_tags = player_soup.find_all('div', {'class': 'webix_ss_body'})[3].find_all('div', {'role': 'gridcell'})
    if len(player_data_tags) % len(player_data_headers) != 0:
        raise Exception('Number of player_data_tags can not be exactly divided by player_Data_header')
    total_matches = int(len(player_data_tags) / len(player_data_headers))
    # player_data_tags has total_matches*29 or total_matches*14 elements, first along matches, then different columns
    #  (data, leagues, etc.) -- need to translate dimension to per match also for non-starting player, IndexError
    print('processing', player_name_df[player_name_df['roto_Player_ID'] == roto_ID]['roto_Name'].iloc[0], 'with', total_matches, 'matches')
    for match_ind in range(0, total_matches):  # loop through all matches
        curr_match_list = []
        for column_ind in range(0, len(player_data_headers)):
            try:
                curr_match_list.append(player_data_tags[match_ind + column_ind * total_matches].contents[0].string)
            except IndexError:  # non starting player contents is empty
                curr_match_list.append(False)
        curr_player_data_list.append(curr_match_list)
        # player_data_tags[match_ind*len(player_data_headers) + column_ind].contents[0]

    # create dataframe for current player
    curr_player_data_df = pd.DataFrame(curr_player_data_list, columns=player_data_headers)

    # Drop data not from World Cup
    curr_player_data_df = curr_player_data_df[curr_player_data_df['League'] == 'World Cup']
    if len(curr_player_data_df) == 0:
        print(player_name_df[player_name_df['roto_Player_ID'] == roto_ID]['roto_Name'].iloc[0], 'has no World Cup data')

    # Include player name and ID in roto
    curr_player_data_df['roto_Player_ID'] = int(roto_ID)
    curr_player_data_df['roto_Name'] = player_name_df[player_name_df['roto_Player_ID'] == roto_ID]['roto_Name'].iloc[0]

    # Get Match_ID
    curr_player_data_df['MatchID'] = 0
    player_team = player_name_df[player_name_df['roto_Player_ID'] == roto_ID]['roto_TeamName'].iloc[0]
    for curr_opp in curr_player_data_df['OPP']:
        if len(curr_opp) == 3:  # player team is the home team if 3 characters in OPP
            curr_opp_full_name = player_name_df[player_name_df['TeamAbbrev'] == curr_opp]['roto_TeamName'].unique()[0]
            match_ID = \
                match_list_df[(match_list_df['Home'] == player_team) & (match_list_df['Away'] == curr_opp_full_name)][
                    'MatchID']
        elif len(curr_opp) == 4:  # player team is the away team if 4 characters in OPP
            curr_opp_full_name = player_name_df[player_name_df['TeamAbbrev'] == curr_opp[1:]]['roto_TeamName'].unique()[
                0]
            match_ID = \
                match_list_df[(match_list_df['Away'] == player_team) & (match_list_df['Home'] == curr_opp_full_name)][
                    'MatchID']
        else:
            raise Exception('OPP not 3 or 4 chars -- something wrong')
        if len(match_ID) != 1:  # check if match found in DB
            raise Exception('Does not find Match ID or Found multiple')
        curr_player_data_df.loc[curr_player_data_df['OPP'] == curr_opp, 'MatchID'] = match_ID.iloc[0]

    # Verify Match_ID by matching game date
    for Date, match_ID in zip(curr_player_data_df['Date'], curr_player_data_df['MatchID']):
        match_date_roto = datetime.strptime(Date, '%b %d')
        match_date_MatchList = match_list_df[match_list_df['MatchID'] == match_ID]['Date'].iloc[0]
        if (match_date_roto.day != match_date_MatchList.day) | (match_date_roto.month != match_date_MatchList.month):
            raise Exception('date not matched in MatchList and roto -- check Match_ID')

    # Move Player_ID and Match_ID to the front of df
    cols = curr_player_data_df.columns.tolist()
    cols = cols[-3:] + cols[:-3]
    curr_player_data_df = curr_player_data_df[cols]

    # Write to df every player
    if is_goally:
        player_data_goalkeeper_df = player_data_goalkeeper_df.append(curr_player_data_df, ignore_index=True, sort=False)
        player_data_goalkeeper_df.to_sql('PlayerData_GoalKeeper', disk_engine, index=False, if_exists='replace')
    else:
        player_data_df = player_data_df.append(curr_player_data_df, ignore_index=True, sort=False)
        player_data_df.to_sql('PlayerData', disk_engine, index=False, if_exists='replace')


roto player Fyodor Smolov already in player_data_df, continue
roto player Aleksandr Samedov already in player_data_df, continue
roto player Artem Dzyuba already in player_data_df, continue

working on roto player Denis Cheryshev
processing Denis Cheryshev with 17 matches

working on roto player Aleksey Miranchuk
processing Aleksey Miranchuk with 11 matches

working on roto player Aleksandr Golovin
processing Aleksandr Golovin with 17 matches

working on roto player Yuri Zhirkov


WebDriverException: Message: Failed to decode response from marionette
